In [ ]:
# !pip install langchain==0.0.125 openai==0.27.2 chromadb==0.3.14 pypdf==3.7.0 tiktoken==0.3.3 gradio==3.23

In [1]:
import os
import openai
import pypdf
import gradio as gr

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA, RetrievalQA

from langchain.document_loaders import TextLoader, PyPDFLoader
import tiktoken


In [2]:
openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
import os

os.listdir("../data/pdfs/")

['Radio.pdf',
 'Progress.pdf',
 'load.pdf',
 'Markdown.pdf',
 'Textbox.pdf',
 'Code.pdf',
 'Warning.pdf',
 'Blocks.pdf',
 'Info.pdf',
 'Slider.pdf',
 'BarPlot.pdf',
 'Dataset.pdf',
 'Flagging.pdf',
 'ChatInterfaceNEW.pdf',
 'JSON.pdf',
 'Themes.pdf',
 'Number.pdf',
 'Group.pdf',
 'Gallery.pdf',
 'DuplicateButton.pdf',
 'AnnotatedImage.pdf',
 'Interface.pdf',
 'TabbedInterface.pdf',
 'File.pdf',
 'UploadButton.pdf',
 'Label.pdf',
 'Accordion.pdf',
 'EventData.pdf',
 'Examples.pdf',
 'LogoutButton.pdf',
 'Tab.pdf',
 'Error.pdf',
 'ScatterPlot.pdf',
 'CheckboxGroup.pdf',
 'HighlightedText.pdf',
 'Request.pdf',
 'State.pdf',
 'Row.pdf',
 'mount_gradio_app.pdf',
 'Audio.pdf',
 'ClearButton.pdf',
 'Dropdown.pdf',
 'LoginButton.pdf',
 'ImageEditor.pdf',
 'Video.pdf',
 'HTML.pdf',
 'Dataframe.pdf',
 'LinePlot.pdf',
 'Job.pdf',
 'Column.pdf',
 'Plot.pdf',
 'Checkbox.pdf',
 'ColorPicker.pdf',
 'Button.pdf',
 'make_waveform.pdf',
 'Chatbot.pdf',
 'Client.pdf',
 'Image.pdf',
 'FileExplorer.pdf',
 

In [3]:
loader = PyPDFLoader("../data/docs/Let's_Build_Your_GPT_Chatbot_with_Gradio.pdf")
documents = loader.load()

In [12]:
import pprint
pprint.pprint(documents[0])
# documents[0]
documents[0].metadata

Document(page_content="2023/11/22 10:55 Let's Build Your GPT Chatbot with Gradio - Analytics Vidhya\nhttps://www.analyticsvidhya.com/blog/2023/05/lets-build-your-gpt-chatbot-with-gradio/#h-gradio-block 1/13Home\nSunil Kumar Dash — Updated On September 14th, 2023\nBeginner chatbot ChatGPT Generative AI Machine LearningMaster the Art of Data Science: A Step-by-Step Guide to Becoming an Industry-Ready Data ScientistDownload Roadmap\n×\nIntroduction\nEver since OpenAI launched ChatGPT, the internet hasn’t stopped speculating about the future of technology or\nhumanity in general. ChatGPT has emerged as a revolutionary product that has the potential to impact almost every\nsphere of human work. OpenAI also has released APIs for the underlying models of chatGPT, “gpt-3.5-turbo” and “gpt-\n4” (currently on the waitlist). For developers, integrating these APIs represents a new frontier of innovation. In this\narticle, we will build GPT chatbot with Gradio and OpenAI chat models.\nLearning Obje

{'source': "../data/docs/Let's_Build_Your_GPT_Chatbot_with_Gradio.pdf",
 'page': 0}

In [25]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-3.5-turbo"), chain_type="stuff", retriever=vectordb.as_retriever())

In [26]:
# プロンプトの定義
from langchain import PromptTemplate

template = """
私はソフトウェアの開発者で、python, gradioを利用しソフトウェアを開発したいと思います。
あなたは親切なアシスタントです。ステップバイステップで、下記の質問に日本語で回答してください。
質問：{question}
回答：
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template,
)

In [28]:
# 
import gradio as gr

def add_text(history, text):
    history = history + [(text, None)]
    return history, ""

def bot(history):
    query = history[-1][0]
    query = prompt.format(question=query)
    answer = qa.run(query)
    source = qa._get_docs(query)[0]
    source_sentence = source.page_content
    answer_source = source_sentence +"\n"+"source:"+source.metadata["source"] + ", page:" + str(source.metadata["page"])
    history[-1][1] = answer # + "\n\n情報ソースは以下です：\n" + answer_source
    return history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot([], elem_id="chatbot")

    with gr.Row():
        with gr.Column(scale=0.6):
            txt = gr.Textbox(
                show_label=False,
                placeholder="Enter text and press enter",
            )

    txt.submit(add_text, [chatbot, txt], [chatbot, txt]).then(
        bot, chatbot, chatbot
    )

demo.launch()
  

/Users/nakashima_toshio/PycharmProjects/gpt4all_langchain_fastapi_gradio/venv/lib/python3.11/site-packages/gradio/layouts/column.py:55: UserWarning: 'scale' value should be an integer. Using 0.6 will cause issues.
  warnings.warn(


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
